In [1]:
import os

In [2]:
DATA_DIR = './text'
labels = [
    x
    for x in os.listdir(DATA_DIR)
    if os.path.isdir(os.path.join(DATA_DIR, x))
]
data_files = {
    label: [
        file
        for file in os.listdir(os.path.join(DATA_DIR, label))
        if file != 'LICENSE.txt'
    ]
    for label in labels
}

In [3]:
def read_file(file: str) -> dict:
    with open(file) as f:
        s = f.read()
    l = s.split('\n')
    return {
        'url': l[0],
        'date': l[1],
        'title': l[2],
        'text': '\n'.join(l[3:])
    }

In [4]:
from collections import ChainMap
import pandas as pd

In [5]:
data_list = [
    ChainMap(
        read_file(os.path.join(DATA_DIR, label, file)),
        {
            'id': os.path.splitext(file)[0],
            'label': label
        }
    )
    for label in labels
    for file in data_files[label]
]
df = pd.DataFrame(data_list, columns=['id', 'label', 'url', 'date', 'title', 'text'])

In [6]:
df.head()

,id,label,url,date,title,text
0,movie-enter-6352654,movie-enter,http://news.livedoor.com/article/detail/6352654/,2012-03-09T11:50:00+0900,「家政婦のミタ」の忽那汐里が声優に初挑戦、健気な妹を演じる,7月7日公開映画『グスコーブドリの伝記』にてメイン声優として、主役のブドリ役に小栗旬、ブド...
1,movie-enter-6462498,movie-enter,http://news.livedoor.com/article/detail/6462498/,2012-04-13T10:00:00+0900,【特別映像】凶暴な巨大生物が救世主を公開処刑,今から100年前の発表以来、数々の大作映画に影響を与えてきたといわれる伝説的小説「火星のプ...
2,movie-enter-6141525,movie-enter,http://news.livedoor.com/article/detail/6141525/,2011-12-23T13:00:00+0900,“日本のトップ総取替え”発言が現実になった＜2011年インタビューまとめ vol.3＞,ムービーエンターの今年行った俳優・女優の取材を一挙に紹介する「2011年インタビューまとめ...
3,movie-enter-5900294,movie-enter,http://news.livedoor.com/article/detail/5900294/,2011-09-30T05:00:00+0900,マツコ・デラックス、世界のセレブに対しても容赦なし,ある日突然、未来を予見するような奇妙なヴィジョンに翻弄され始める弁護士イーライの摩訶不思議...
4,movie-enter-6371410,movie-enter,http://news.livedoor.com/article/detail/6371410/,2012-03-15T13:36:00+0900,【いちおう妖ヶ劇場】第9話：生徒を洗脳するなんて、あんまりせんのうの巻,元お嬢様から貧乏に転落した勅使河原 栄華（てしがわら えいか）は、映画好きのネコ仙人ら妖怪...


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

In [8]:
sss = StratifiedShuffleSplit(1)
teacher_index, test_index = next(sss.split(df, df.label))

In [9]:
teacher_df = df.iloc[teacher_index]
test_df = df.iloc[test_index]

In [10]:
from normalize_neologd import normalize_neologd

In [11]:
teacher_df['normalized_title'] = teacher_df.apply(lambda x: normalize_neologd(x.title), axis=1)
teacher_df['normalized_text'] = teacher_df.apply(lambda x: normalize_neologd(x.text), axis=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
teacher_df.head()

,id,label,url,date,title,text,normalized_title,normalized_text
6995,it-life-hack-6517094,it-life-hack,http://news.livedoor.com/article/detail/6517094/,2012-04-29T20:00:00+0900,こんこん！紺子には秘策があるんだからねっ【紺子にゅうす】,こんこん♪\nふぉっくす紺子です☆\n \n今回の紺子にゅうすは\nascii.jp（htt...,こんこん!紺子には秘策があるんだからねっ【紺子にゅうす】,こんこん♪\nふぉっくす紺子です☆\n \n今回の紺子にゅうすは\nascii.jp(htt...
5665,smax-6521195,smax,http://news.livedoor.com/article/detail/6521195/,2012-05-01T16:55:00+0900,ソフトバンク、5月7日からソフトバンク携帯電話取扱店でもらえる「ローマかぶれグッズ」プレゼン...,ステッカーとマグネット気に入るのはどっち？ \n\nソフトバンクモバイルは4月27日、ソフト...,ソフトバンク、5月7日からソフトバンク携帯電話取扱店でもらえる「ローマかぶれグッズ」プレゼン...,ステッカーとマグネット気に入るのはどっち?\n\nソフトバンクモバイルは4月27日、ソフトバ...
119,movie-enter-6398017,movie-enter,http://news.livedoor.com/article/detail/6398017/,2012-03-24T15:00:00+0900,死ぬまで戦え！ 『スターシップ・トゥルーパーズ』昆虫と人類の壮絶な戦いが再び,SF小説の巨匠ロバート・A・ハインラインの傑作「宇宙の戦士」の映画化にして、クエンティン・...,死ぬまで戦え!『スターシップ・トゥルーパーズ』昆虫と人類の壮絶な戦いが再び,SF小説の巨匠ロバート・A・ハインラインの傑作「宇宙の戦士」の映画化にして、クエンティン・タ...
541,movie-enter-6399270,movie-enter,http://news.livedoor.com/article/detail/6399270/,2012-03-24T16:19:00+0900,DAIGO、33歳でウルトラマンにオファーされたのは想定外,ウルトラマンシリーズ最新作『ウルトラマンサーガ』が24日より全国公開となった。今回、ウルト...,DAIGO、33歳でウルトラマンにオファーされたのは想定外,ウルトラマンシリーズ最新作『ウルトラマンサーガ』が24日より全国公開となった。今回、ウルトラ...
297,movie-enter-6178173,movie-enter,http://news.livedoor.com/article/detail/6178173/,2012-01-10T15:45:00+0900,究極のヴァンパイア・ハンター、“戦う聖職者”の聖書に隠された武器,1892年に出版された、アイルランド人作家ブラム・ストーカーの小説「ドラキュラ」に登場する...,"究極のヴァンパイア・ハンター、“戦う聖職者""の聖書に隠された武器",1892年に出版された、アイルランド人作家ブラム・ストーカーの小説「ドラキュラ」に登場する男...


In [13]:
from janome.tokenizer import Tokenizer

In [14]:
tokenizer = Tokenizer(mmap=True)

In [17]:
def select_pos(token: 'Token') -> bool:
    return token.part_of_speech.split(',')[0] in ['名詞', '動詞', '形容詞']

In [19]:
with open('./stop_words.txt') as f:
    STOP_WORDS = f.readlines()
def stop_words(token: 'Token') -> bool:
    return token.surface in STOP_WORDS

In [20]:
def select_token(tokens: list) -> list:
    return [
        token.surface
        for token in tokens
        if select_pos(token)
        if not stop_words(token)
    ]

In [23]:
teacher_df['wakati'] = teacher_df.text.apply(lambda x: select_token(tokenizer.tokenize(x)))

KeyboardInterrupt: 